In [8]:
import yt_dlp

def download_video(url):
    ydl_opts = {
        'format': 'best[ext=mp4]',
        'outtmpl': 'a.%(ext)s',
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    
    print("Descarga completada: a.mp4")

url = "https://www.youtube.com/watch?v=NjjeO5eAJD4"
download_video(url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=NjjeO5eAJD4
[youtube] NjjeO5eAJD4: Downloading webpage
[youtube] NjjeO5eAJD4: Downloading tv simply player API JSON
[youtube] NjjeO5eAJD4: Downloading tv client config
[youtube] NjjeO5eAJD4: Downloading tv player API JSON
[info] NjjeO5eAJD4: Downloading 1 format(s): 18
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: a.mp4
[download] 100% of    2.31MiB in 00:00:01 at 1.17MiB/s   
Descarga completada: a.mp4


In [10]:
import cv2
import numpy as np

def capture_frames_per_second(video_path):
    cap = cv2.VideoCapture(video_path)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    frame_count = 0
    fps_interval = int(round(fps))
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        if frame_count % fps_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
            
        frame_count += 1
    
    cap.release()
    return frames

def create_comparison_video(frames, output_path="comparison.mp4"):
    if len(frames) < 2:
        print("Se necesitan al menos 2 frames")
        return
    
    # Calcular diferencias
    differences = []
    for i in range(1, len(frames)):
        frame_current = frames[i].astype(np.float32)
        frame_previous = frames[i-1].astype(np.float32)
        
        difference = np.abs(frame_current - frame_previous)
        difference = np.clip(difference, 0, 255).astype(np.uint8)
        differences.append(difference)
    
    # Convertir frames originales a escala de grises
    gray_frames = []
    for frame in frames[1:]:  # Empezar desde frame 2
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
        gray_frames.append(gray_rgb)
    
    # Crear video de comparación lado a lado
    height, width = frames[0].shape[:2]
    combined_width = width * 2
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 1.0, (combined_width, height))
    
    for i in range(len(differences)):
        # Frame original en escala de grises (izquierda)
        left_frame = gray_frames[i]
        
        # Diferencia (derecha)
        right_frame = differences[i]
        
        # Combinar lado a lado
        combined_frame = np.hstack([left_frame, right_frame])
        
        # Convertir RGB a BGR para OpenCV
        combined_bgr = cv2.cvtColor(combined_frame, cv2.COLOR_RGB2BGR)
        
        out.write(combined_bgr)
        print(f"Procesando frame {i+2}")
    
    out.release()
    print(f"Video guardado como: {output_path}")

video_path = "a.mp4"
frames = capture_frames_per_second(video_path)
print(f"Frames capturados: {len(frames)}")

create_comparison_video(frames)

Frames capturados: 30
Procesando frame 2
Procesando frame 3
Procesando frame 4
Procesando frame 5
Procesando frame 6
Procesando frame 7
Procesando frame 8
Procesando frame 9
Procesando frame 10
Procesando frame 11
Procesando frame 12
Procesando frame 13
Procesando frame 14
Procesando frame 15
Procesando frame 16
Procesando frame 17
Procesando frame 18
Procesando frame 19
Procesando frame 20
Procesando frame 21
Procesando frame 22
Procesando frame 23
Procesando frame 24
Procesando frame 25
Procesando frame 26
Procesando frame 27
Procesando frame 28
Procesando frame 29
Procesando frame 30
Video guardado como: comparison.mp4
